### Define Preprocessing, Model, Postprocessing

In [5]:
import subprocess
import tempfile
import gradio
import base64
import numpy as np
from io import BytesIO

MODEL_PATH = "../exp0/models/model_step_2000.pt"
OUTPUT_FILE = "pred_train.txt"

'''
The microphone input sends a base64 representation of a wav file to the backend, so this should be
decoded and saved into a file object that can be used by the translate.py script. The script below
should do that.
'''
def preprocessing(inp):
    file_obj = gradio.preprocessing_utils.decode_base64_to_wav_file(inp)
    return file_obj

'''
The model function here simply calls the `translate.py` script with the appropriate command line
arguments. I was unable to test this since I ran into errors using opennmt.
'''
def model(file_obj):
    src_file = tempfile.NamedTemporaryFile()
    src_file.close()
    with open(src_file.name, 'w') as f:
        f.write(file_obj.name)
    
    subprocess.call(f"python translate.py -data_type audio "
                    f"-model {MODEL_PATH} "
                    f"-src_dir / -src {src_file.name} "
                    f"-output {OUTPUT_FILE} -verbose", shell=True)
    
    with open(src_file.name, 'r') as f:
        first_line = f.readline()
    return first_line
    
'''
The textbox interface displays a string. I don't think any additional steps need to be done to postprocess 
the interface, but I could be mistaken since I couldn't get translate.py working
'''
def postprocessing(out):
    return 

### Launch `Gradio` Interface 

In [6]:
inp = gradio.inputs.Microphone(preprocessing_fn=preprocessing)
out = gradio.outputs.Textbox(postprocessing_fn=postprocessing)
io = gradio.Interface(inputs=inp, outputs=out, model_type="pyfunc", model=model)
io.launch(validate=False, share=False)

AttributeError: module 'gradio.inputs' has no attribute 'Microphone'